In [1]:
CODE_DIR = '/home/shreyas/Documents/git/wheelai/'
#DATA_DIR = '/media/shreyas/DATA/ML_DATA/wheelai/gtaV/'
DATA_DIR = '/media/shreyas/DATA/ML_DATA/wheelai/gtaV/sample/'
traindata_path = DATA_DIR + 'train/'
validdata_path = '/media/shreyas/DATA/ML_DATA/wheelai/gtaV/valid/'
results_path = '/media/shreyas/DATA/ML_DATA/wheelai/gtaV/results/'

In [2]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model,  Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing import image
from keras.applications import VGG16
from keras.layers import Lambda, Cropping2D

%matplotlib inline



Using TensorFlow backend.


### Finetune and Train

In [3]:
batch_size = 32
epochs = 3

In [4]:
def get_batches(path, class_mode='categorical', gen=image.ImageDataGenerator(rescale=1./255), \
                shuffle=True, target_size=(224,224), batch_size=1):
    return gen.flow_from_directory(path, class_mode=class_mode, batch_size=batch_size, \
                                   target_size=target_size, shuffle=shuffle)

def get_steps(batches, batch_size):
    steps = int(batches.samples/batch_size)
    return (steps if batches.samples%batch_size==0 else (steps+1))


In [5]:
train_b = get_batches(traindata_path, batch_size=batch_size)
valid_b = get_batches(validdata_path, batch_size=batch_size)

Found 3006 images belonging to 3 classes.
Found 554 images belonging to 3 classes.


In [6]:
train_steps = get_steps(train_b, batch_size)
valid_steps = get_steps(valid_b, batch_size)
num_class = train_b.num_class

In [7]:
train_labels = train_b.classes
valid_labels = valid_b.classes

In [8]:
y_train = keras.utils.to_categorical(train_labels)
y_valid = keras.utils.to_categorical(valid_labels)

In [9]:
print(train_labels.shape, valid_labels.shape, y_train.shape, y_valid.shape)

(3006,) (554,) (3006, 3) (554, 3)


In [10]:
trn_b = get_batches(traindata_path, class_mode=None, shuffle=False, batch_size=batch_size)
val_b = get_batches(validdata_path, class_mode=None, shuffle=False, batch_size=batch_size)

Found 3006 images belonging to 3 classes.
Found 554 images belonging to 3 classes.


In [11]:
gen_augdata = image.ImageDataGenerator(width_shift_range=.2, height_shift_range=.2, 
                                      shear_range=0.2, zoom_range=0.2)
augtrain_b = get_batches(traindata_path, gen=gen_augdata, class_mode='categorical',\
                         batch_size=batch_size)

Found 3006 images belonging to 3 classes.


In [16]:
def vgg_conv():
    model = VGG16(include_top=False, weights='imagenet', input_shape=(122,224,3))
    layers = model.layers
    last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Conv2D][-1]
    conv_layers = layers[:15]
    conv_model = Sequential(conv_layers)
    model = Sequential()
    model.add(Cropping2D(cropping=((73,29), (0,0)), input_shape=(224, 224, 3)))
    model.add(conv_model)
    return model


def top_layer(input_shape):
    model = Sequential()
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', input_shape=input_shape))
    model.add(Dropout(0.1))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Dropout(0.1))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(2,2))
    model.add(Flatten())
    model.add(Dense(256, activation='elu'))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='elu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_class, activation='softmax'))
    
    return model 

In [17]:
def basic_model(input_shape):
    model = Sequential()
    #model.add(BatchNormalization(axis=1, input_shape=input_shape))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu',input_shape=input_shape))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(Flatten())    
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(3, activation='softmax'))
    return model

In [18]:
def get_ft(trn_b, train_steps, val_b, valid_steps):
    model = vgg_conv()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    trn_ft = model.predict_generator(trn_b, train_steps)
    val_ft = model.predict_generator(val_b, valid_steps)
    print(trn_ft.shape, val_ft.shape)

    np.save(results_path + 'trn_ft.npy', trn_ft)
    np.save(results_path + 'val_ft.npy', val_ft)

In [19]:
get_ft(trn_b, train_steps, val_b, valid_steps)

(3006, 7, 14, 512) (554, 7, 14, 512)


In [20]:
X_train = np.load(results_path + 'trn_ft.npy')
X_valid = np.load(results_path + 'val_ft.npy')

In [21]:
model = top_layer(X_train.shape[1:])

In [22]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=2e-5), metrics=['accuracy'])

In [ ]:
#vgg_model.fit(X_train, y_train, epochs=2, batch_size=1, verbose=1, \
#          validation_data=(X_valid, y_valid))
#vgg_model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=10,\
#                   validation_data=valid_b, validation_steps=valid_steps)

In [ ]:
#model.save_weights(results_path+'trial_bottleneck.h5')

In [23]:
filepath1 = results_path+'vgg_bottleneck.h5'
checkpoint1 = ModelCheckpoint(filepath1, monitor='val_loss', verbose=1, save_best_only=True,\
                             save_weights_only=True, mode='min', period=1)
callbacks1=[checkpoint1]

In [25]:
model.fit(X_train, y_train, epochs=21, batch_size=1, verbose=1, \
          callbacks=callbacks1, validation_data=(X_valid, y_valid))

Train on 3006 samples, validate on 554 samples
Epoch 1/21
3006/3006 [==============================] - 52s - loss: 0.4812 - acc: 0.8007 - val_loss: 0.5310 - val_acc: 0.7690
Epoch 2/21
3006/3006 [==============================] - 52s - loss: 0.3739 - acc: 0.8599 - val_loss: 0.5149 - val_acc: 0.8014
Epoch 3/21
3006/3006 [==============================] - 53s - loss: 0.2715 - acc: 0.8892 - val_loss: 0.4834 - val_acc: 0.8087
Epoch 4/21
3006/3006 [==============================] - 52s - loss: 0.2090 - acc: 0.9248 - val_loss: 0.5115 - val_acc: 0.8051
Epoch 5/21
3006/3006 [==============================] - 53s - loss: 0.1635 - acc: 0.9398 - val_loss: 0.4698 - val_acc: 0.8249
Epoch 6/21
3006/3006 [==============================] - 53s - loss: 0.1219 - acc: 0.9571 - val_loss: 0.5961 - val_acc: 0.7942
Epoch 7/21
3006/3006 [==============================] - 52s - loss: 0.1122 - acc: 0.9611 - val_loss: 0.6004 - val_acc: 0.7906
Epoch 8/21
3006/3006 [==============================] - 52s - loss: 0.0

In [26]:
def vgg_model():
    base_model = vgg_conv()
    for layer in base_model.layers[:11]: layer.trainable=False
    print (base_model.output_shape[1:])
    top_model = top_layer(base_model.output_shape[1:])
    top_model.load_weights(results_path+'vgg_bottleneck.h5')
    
    base_model.add(top_model)
    
    return base_model

In [27]:
vgg_model = vgg_model()
vgg_model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=2e-5, momentum=0.9), metrics=['accuracy'])

(7, 14, 512)


In [28]:
filepath2 = results_path+'vgg_ft.h5'
checkpoint2 = ModelCheckpoint(filepath2, monitor='val_loss', verbose=1,\
                              save_best_only=True, mode='min', period=1)
callbacks2=[checkpoint2]

In [29]:
vgg_model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=21, callbacks=callbacks2,\
                        validation_data=valid_b, validation_steps=valid_steps)

Epoch 1/21
94/94 [==============================] - 50s - loss: 0.0905 - acc: 0.9704 - val_loss: 0.4617 - val_acc: 0.8303
Epoch 2/21
94/94 [==============================] - 45s - loss: 0.0723 - acc: 0.9790 - val_loss: 0.4604 - val_acc: 0.8285
Epoch 3/21
94/94 [==============================] - 44s - loss: 0.0690 - acc: 0.9790 - val_loss: 0.4797 - val_acc: 0.8177
Epoch 4/21
94/94 [==============================] - 44s - loss: 0.0637 - acc: 0.9843 - val_loss: 0.4690 - val_acc: 0.8267
Epoch 5/21
94/94 [==============================] - 45s - loss: 0.0630 - acc: 0.9814 - val_loss: 0.4152 - val_acc: 0.8448
Epoch 6/21
94/94 [==============================] - 45s - loss: 0.0611 - acc: 0.9824 - val_loss: 0.4134 - val_acc: 0.8357
Epoch 7/21
94/94 [==============================] - 44s - loss: 0.0558 - acc: 0.9864 - val_loss: 0.5112 - val_acc: 0.8177
Epoch 8/21
94/94 [==============================] - 45s - loss: 0.0543 - acc: 0.9844 - val_loss: 0.4092 - val_acc: 0.8285
Epoch 9/21
94/94 [======

In [30]:
filepath3 = results_path+'vgg_augmented_ft.h5'
checkpoint3 = ModelCheckpoint(filepath3, monitor='val_loss', verbose=1,\
                              save_best_only=True, mode='min', period=1)
callbacks3=[checkpoint3]

In [31]:
vgg_model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=13, callbacks=callbacks3,\
                        validation_data=valid_b, validation_steps=valid_steps)

Epoch 1/13
94/94 [==============================] - 62s - loss: 7.5185 - acc: 0.5173 - val_loss: 0.5042 - val_acc: 0.8177
Epoch 2/13
94/94 [==============================] - 56s - loss: 7.1695 - acc: 0.5313 - val_loss: 0.6675 - val_acc: 0.7708
Epoch 3/13
94/94 [==============================] - 55s - loss: 6.9550 - acc: 0.5429 - val_loss: 0.5594 - val_acc: 0.7906
Epoch 4/13
94/94 [==============================] - 55s - loss: 6.4661 - acc: 0.5622 - val_loss: 0.6848 - val_acc: 0.7401
Epoch 5/13
94/94 [==============================] - 55s - loss: 6.4599 - acc: 0.5673 - val_loss: 0.5704 - val_acc: 0.7798
Epoch 6/13
94/94 [==============================] - 56s - loss: 5.9460 - acc: 0.5915 - val_loss: 0.5133 - val_acc: 0.7888
Epoch 7/13
94/94 [==============================] - 57s - loss: 5.5015 - acc: 0.6047 - val_loss: 0.4690 - val_acc: 0.7834
Epoch 8/13
94/94 [==============================] - 56s - loss: 5.3650 - acc: 0.5998 - val_loss: 0.5514 - val_acc: 0.7726
Epoch 9/13
94/94 [======

In [ ]:
model.save_weights(results_path+'vgg_ft_augmenteddata.h5')

In [ ]:
preds.shape

In [ ]:
print (preds[:5])
img = batches.filenames
print (img[:5])

In [ ]:
from PIL import Image
Image.open(DATA_DIR+'test/'+img[2])

In [ ]:
save_array(results_path + 'test_predictions.dat', preds)
save_array(results_path + 'imagefiles.dat', img)

### Validate Predictions
Lets plot -
1. A few correct labels at random
2. A few incorrect labels at random
3. Most confident correct predictions of each class
4. Most confident incorrect predictions of each class
5. Most uncertain labels (probabilites close to 0.5)

In [ ]:
vgg.model.load_weights(filepath)

In [ ]:
val_batches, probs = vgg.test(validdata_path, batch_size=batch_size*2)

In [ ]:
img = val_batches.filenames
expected_labels = val_batches.classes

our_predictions = probs[:,0]
other_predictions = np.round(probs[:,1])
our_labels = np.round(1-our_predictions)

In [ ]:
from keras.preprocessing import image

def plots_idx(idx, titles=None):
    plots([image.load_img(validdata_path + img[i]) for i in idx], titles=titles)

In [ ]:
n_view = 4

In [ ]:
correct = np.where(our_labels==expected_labels)[0]
print ("Found %d correct labels" % len(correct))
idx = permutation(correct)[:n_view]
plots_idx(idx, our_predictions[idx])

In [ ]:
incorrect = np.where(our_labels!=expected_labels)[0]
print ("Found %d incorrect labels" % len(incorrect))
idx = permutation(incorrect)[:n_view]
plots_idx(idx, our_predictions[idx])

In [ ]:

#3a. The images we most confident were cats, and are actually cats
correct_cats = np.where((our_labels==0) & (our_labels==expected_labels))[0]
print ("Found %d confident correct cats labels" % len(correct_cats))
most_correct_cats = np.argsort(our_predictions[correct_cats])[::-1][:n_view]
plots_idx(correct_cats[most_correct_cats], our_predictions[correct_cats][most_correct_cats])

In [ ]:
#3b. The images we most confident were dogs, and are actually dogs
correct_dogs = np.where((our_labels==1) & (our_labels==expected_labels))[0]
print ("Found %d confident correct dogs labels" % len(correct_dogs))
most_correct_dogs = np.argsort(our_predictions[correct_dogs])[:n_view]
plots_idx(correct_dogs[most_correct_dogs], our_predictions[correct_dogs][most_correct_dogs])

In [ ]:
#4a. The images we were most confident were cats, but are actually dogs
incorrect_cats = np.where((our_labels==0) & (our_labels!=expected_labels))[0]
print ("Found %d incorrect cats" % len(incorrect_cats))
if len(incorrect_cats):
    most_incorrect_cats = np.argsort(our_predictions[incorrect_cats])[::-1][:n_view]
    plots_idx(incorrect_cats[most_incorrect_cats], our_predictions[incorrect_cats][most_incorrect_cats])

In [ ]:
#4b. The images we were most confident were dogs, but are actually cats
incorrect_dogs = np.where((our_labels==1) & (our_labels!=expected_labels))[0]
print ("Found %d incorrect dogs" % len(incorrect_dogs))
if len(incorrect_dogs):
    most_incorrect_dogs = np.argsort(our_predictions[incorrect_dogs])[:n_view]
    plots_idx(incorrect_dogs[most_incorrect_dogs], our_predictions[incorrect_dogs][most_incorrect_dogs])

In [ ]:
#5. The most uncertain labels (ie those with probability closest to 0.5).
most_uncertain = np.argsort(np.abs(our_predictions-0.5))
plots_idx(most_uncertain[:n_view], our_predictions[most_uncertain])

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(expected_labels, our_labels)
plot_confusion_matrix(cm, val_batches.class_indices)

In [ ]:
preds = load_array(results_path + 'test_predictions.dat')
filenames = load_array(results_path + 'imagefiles.dat')

In [ ]:
isdog = preds[:,1]
print ("Raw Predictions: "+ str(isdog[:5]))
print ("Mid Predictions: "+str(isdog[(isdog<.6)&(isdog>0.4)]))
print ("Edge Predictions: "+str(isdog[(isdog<0.02)&(isdog>.98)]))

In [ ]:
np.amax(isdog)

In [ ]:
#Extract imageIds from the filenames in our test/unknown directory 
filenames = batches.filenames
ids = np.array([int(f[8:f.find('.')]) for f in filenames])

In [ ]:
subm = np.stack([ids,isdog], axis=1)
subm[:5]

In [ ]:
%cd $DATA_DIR
submission_file_name = 'submission1.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')